<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
# TODO - Words, words, mere words, no matter from the heart.

In [2]:
file='./100-0.txt'

In [3]:
with open(file, 'r') as data:
    content=data.read()
sonnets=content.split('\n')[139:2907]
plays=content.split('\n')[2907:166478]

In [4]:
print(len(sonnets))
for sonn in sonnets:
    try:
        int(sonn.strip())
        sonnets.remove(sonn)
    except:
        pass
print(sum([len(i) for i in sonnets])/len(sonnets))
len(sonnets)


2768
35.22982791586998


2615

In [5]:
#maintain the newlines that were taken out before
sonnets='\n'.join(sonnets)
#sonnets=sonnets.lower()

In [22]:
maxlen=35
#thats an even divisor of 35 thats small, I have no idea how this will impact things but seems relevant
step=3
sentences=[]
next_chars=[]

for i in range(0, len(sonnets)-maxlen, step):
    sentences.append(sonnets[i:i+maxlen])
    next_chars.append(sonnets[maxlen+i])
len(sentences)

31569

In [23]:
sentences[0]

'From fairest creatures we desire in'

In [24]:
chars=sorted(list(set(sonnets)))
char_index=dict((c,i) for i, c in enumerate(chars))
index_char=dict((i,c) for i, c in enumerate(chars))

In [25]:
len(chars)

62

In [26]:
import numpy as np

In [27]:
X=np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y=np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [28]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_index[char]]=1
    y[i, char_index[next_chars[i]]]=1

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import LambdaCallback
import random
import sys
import os

In [40]:
model=Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(256, activation='sigmoid'))
model.add(Dense(len(chars), activation='softmax'))
optimizer=RMSprop(learning_rate=.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [41]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [42]:
char_index

{'\n': 0,
 ' ': 1,
 '!': 2,
 '(': 3,
 ')': 4,
 ',': 5,
 '-': 6,
 '.': 7,
 ':': 8,
 ';': 9,
 '?': 10,
 'A': 11,
 'B': 12,
 'C': 13,
 'D': 14,
 'E': 15,
 'F': 16,
 'G': 17,
 'H': 18,
 'I': 19,
 'J': 20,
 'K': 21,
 'L': 22,
 'M': 23,
 'N': 24,
 'O': 25,
 'P': 26,
 'R': 27,
 'S': 28,
 'T': 29,
 'U': 30,
 'V': 31,
 'W': 32,
 'Y': 33,
 'a': 34,
 'b': 35,
 'c': 36,
 'd': 37,
 'e': 38,
 'f': 39,
 'g': 40,
 'h': 41,
 'i': 42,
 'j': 43,
 'k': 44,
 'l': 45,
 'm': 46,
 'n': 47,
 'o': 48,
 'p': 49,
 'q': 50,
 'r': 51,
 's': 52,
 't': 53,
 'u': 54,
 'v': 55,
 'w': 56,
 'x': 57,
 'y': 58,
 'z': 59,
 '‘': 60,
 '’': 61}

In [43]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(sonnets) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = sonnets[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_index[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = index_char[next_index]
            
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [44]:
next_chars

['c',
 'a',
 ',',
 'h',
 ' ',
 'e',
 'b',
 'b',
 'u',
 '’',
 'r',
 'e',
 'i',
 't',
 'e',
 'r',
 'i',
 '\n',
 't',
 's',
 'h',
 'r',
 'e',
 's',
 'u',
 ' ',
 ' ',
 'm',
 'd',
 'e',
 'e',
 'H',
 ' ',
 'n',
 'r',
 'e',
 ' ',
 'g',
 ' ',
 'a',
 'h',
 ' ',
 'm',
 'y',
 'B',
 ' ',
 'o',
 'c',
 't',
 'c',
 'd',
 'o',
 'h',
 'e',
 'w',
 'b',
 'g',
 ' ',
 'e',
 '\n',
 'e',
 's',
 't',
 ' ',
 'g',
 '’',
 'f',
 'm',
 'w',
 'h',
 'e',
 '-',
 'b',
 'a',
 'i',
 ' ',
 'e',
 '\n',
 'k',
 'g',
 ' ',
 'm',
 'e',
 'h',
 'e',
 'b',
 'd',
 'c',
 'l',
 's',
 'T',
 ' ',
 'l',
 't',
 ' ',
 'e',
 't',
 't',
 ' ',
 'e',
 ' ',
 'l',
 't',
 ' ',
 'u',
 ':',
 'h',
 ' ',
 'a',
 'a',
 ' ',
 'w',
 'h',
 'w',
 'l',
 's',
 'r',
 'h',
 'r',
 'm',
 't',
 'A',
 ' ',
 'l',
 'h',
 'a',
 ' ',
 ' ',
 'e',
 'a',
 'y',
 'p',
 'n',
 '\n',
 't',
 'n',
 'h',
 'e',
 'w',
 'b',
 ' ',
 'r',
 's',
 't',
 ' ',
 'n',
 'n',
 '\n',
 'd',
 't',
 'd',
 ' ',
 'u',
 ',',
 'a',
 's',
 'w',
 't',
 'i',
 'n',
 'g',
 'd',
 'g',
 ' ',
 'i',
 ' ',

In [46]:
model.fit(X, y,
          batch_size=128,
          epochs=5,
          callbacks=[print_callback])

Train on 31569 samples
Epoch 1/5
31488/31569 [============================>.] - ETA: 0s - loss: 1.7050
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "y precious minutes waste,
These vac"
y precious minutes waste,
These vace they the still the time the faire the then the time the can the still the time the time thee thee thee the time be the then the time the then the time thee the still the steak then thee thee the self the still the time the stear the still the time the still the self ther the tand the still the self all the still thy self the time the still the still the see the time the steep then thee the still
----- diversity: 0.5
----- Generating with seed: "y precious minutes waste,
These vac"
y precious minutes waste,
These vace this thee heaven time ther thee seet the day,
When they thee thee and the me the can the still the self the wite,
Thee’ should is the dist all thy pay then devente,
The toot and the tide own then thee and then all

/Users/eyvonnegeordan/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


 hambmars not can have for even mink Lels for arteet a beclence,
Wece bark-ersu   Timed not when nough in thy everyt that will though praise he provine times I divines woed deplate.



So stant in mend my coldentate on my storing the gabled, thou bright,
All the ove,
  Which I have do thee,
and  vew I wombs so carvingr of
----- diversity: 1.2
----- Generating with seed: "gns love and all love’s loving part"
gns love and all love’s loving parttancked but of on thou of from more,
  So shall in his that keep for times lovely then bett now madetben’st I the beauty lebd bloakting:
But peidit,



In they which them faur rid gruder we one thing bul wost; truth?
When dyes re!
y more crobl Tendake bandmed make for by herringrent woild leemerored then in couls inuon deathss and their thou vicht when thou wilt to effared confargeful of the eyely
31569/31569 [==============================] - 218s 7ms/sample - loss: 1.5587
Epoch 4/5
31488/31569 [============================>.] - ETA: 0s - loss: 1.

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN